In [1]:
# %load_ext autoreload
# %autoreload 2

import sys

sys.path.insert(0, "../")

from autogluon.vision import ImagePredictor, ImageDataset
import numpy as np
import pandas as pd
import pickle
import datetime
from pathlib import Path
import cleanlab
from cross_validation_autogluon import cross_val_predict_autogluon_image_dataset

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Read original data

In [2]:
# data path
CIFAR_10_DATA_PATH = "/Data/cifar10_png/"

# read data from root folder
train_dataset, _, test_dataset = \
    ImageDataset.from_folders(
        root=CIFAR_10_DATA_PATH,
    )

## Read data with noisy label

In case cross-validation procedure fails for a model, we want to be able to rerun with the same noisy dataset

In [3]:
noise_amount = 0.2
frac_zero_noise_rates = 0.4

DATA_NOISY_LABELS_OUT_FILE = f"cifar10_train_dataset_noise_amount_{noise_amount}_sparsity_{frac_zero_noise_rates}_20220326055753.csv"

train_dataset_noisy_labels_loaded = pd.read_csv(DATA_NOISY_LABELS_OUT_FILE)

In [4]:
train_dataset_noisy_labels_loaded.head()

,image,label
0,/Data/cifar10_png/train/airplane/0001.png,0
1,/Data/cifar10_png/train/airplane/0002.png,5
2,/Data/cifar10_png/train/airplane/0003.png,9
3,/Data/cifar10_png/train/airplane/0004.png,7
4,/Data/cifar10_png/train/airplane/0005.png,8


In [5]:
train_dataset_noisy_labels_loaded.shape

(50000, 2)

In [12]:
train_dataset_noisy_labels_loaded.groupby("label")["image"].count().reset_index()

,label,image
0,0,3690
1,1,5282
2,2,3693
3,3,5883
4,4,5962
5,5,4411
6,6,5275
7,7,5790
8,8,6030
9,9,3984


## Generate Out-of-Sample Predicted Probabilities via Cross-Validation

In [7]:
%%time

# generate cross-validated predicted probabilities for various models so we can use them for ensemble scoring methods
models = [
    "resnet18",
    "resnet50d",
    "efficientnet_b1",
    "twins_pcpvt_base",
    "swin_base_patch4_window7_224"
]

epochs = 100
holdout_frac = 0.2
n_splits = 5

# run cross-validation for each model
for model in models:
    
    print("----")
    print(f"Running cross-validation for model: {model}")

    MODEL_PARAMS = {
        "model": model,
        "epochs": epochs,
        "holdout_frac": holdout_frac
    }

    # results of cross-validation will be saved to pickle files for each model/fold
    _ = \
        cross_val_predict_autogluon_image_dataset(
            dataset=train_dataset_noisy_labels_loaded, # train with NOISY LABELS
            out_folder=f"./cifar10_train_dataset_noise_amount_{noise_amount}_sparsity_{frac_zero_noise_rates}_cv_{model}/", # save results of cross-validation in pickle files for each fold
            n_splits=n_splits,
            model_params=MODEL_PARAMS
        )

modified configs(<old> != <new>): {
root.misc.seed       42 != 681
root.misc.num_workers 4 != 16
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
root.train.epochs    200 != 100
root.train.early_stop_baseline 0.0 != -inf
root.img_cls.model   resnet101 != swin_base_patch4_window7_224
}
Saved config to /dcai/src/experiments/d7a93a9e/.trial_0/config.yaml


----
Running cross-validation for model: swin_base_patch4_window7_224
----
Running Cross-Validation on Split: 0


Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window7_224_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_base_patch4_window7_224_22kto1k.pth
Model swin_base_patch4_window7_224 created, param count:                                         86753474
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 82.020726 samples/sec	accuracy=0.120000	lr=0.000100
Epoch[0] Batch [99]	Speed: 101.197617 samples/sec	accuracy=0.189375	lr=0.000100
Epoch[0] Batch [149]	Speed: 100.643460 samples/sec	accuracy=0.238333	lr=0.000100
Epoch[0] Batch [199]	Speed: 99.357736 samples/sec	accuracy=0.292187	lr=0.000100
Epoch[0] Batch [249]	Speed: 100.947706 samples/sec	accuracy=0.335750	lr=0.000100
Epoch[0] Batch [299]	Speed: 100.662355 samples/sec	accuracy=0.373333	lr=0.000100
Epoch[0] Batch [349]	Speed: 101.490123 samples/sec	accuracy=0.403393	lr=0.000100
Epoch[0] Batch [

Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_0/_test_pred_probs_split_0
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_0/_test_pred_features_split_0


modified configs(<old> != <new>): {
root.misc.seed       42 != 181
root.misc.num_workers 4 != 16
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
root.train.epochs    200 != 100
root.train.early_stop_baseline 0.0 != -inf
root.img_cls.model   resnet101 != swin_base_patch4_window7_224
}
Saved config to /dcai/src/experiments/e0794446/.trial_0/config.yaml


...

Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_3/_test_pred_probs_split_3
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_3/_test_pred_features_split_3


modified configs(<old> != <new>): {
root.misc.seed       42 != 689
root.misc.num_workers 4 != 16
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
root.train.epochs    200 != 100
root.train.early_stop_baseline 0.0 != -inf
root.img_cls.model   resnet101 != swin_base_patch4_window7_224
}
Saved config to /dcai/src/experiments/2e4940a5/.trial_0/config.yaml


Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_3/_test_labels_split_3
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_3/_test_image_files_split_3
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_3/_test_indices_split_3
----
Running Cross-Validation on Split: 4


Model swin_base_patch4_window7_224 created, param count:                                         86753474
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 80.744072 samples/sec	accuracy=0.153750	lr=0.000100
Epoch[0] Batch [99]	Speed: 101.522552 samples/sec	accuracy=0.224375	lr=0.000100
Epoch[0] Batch [149]	Speed: 100.624150 samples/sec	accuracy=0.279583	lr=0.000100
Epoch[0] Batch [199]	Speed: 100.756329 samples/sec	accuracy=0.320312	lr=0.000100
Epoch[0] Batch [249]	Speed: 99.649503 samples/sec	accuracy=0.360000	lr=0.000100
Epoch[0] Batch [299]	Speed: 100.060711 samples/sec	accuracy=0.397292	lr=0.000100
Epoch[0] Batch [349]	Speed: 100.425632 samples/sec	accuracy=0.427500	lr=0.000100
Epoch[0] Batch [399]	Speed: 100.729933 samples/sec	accuracy=0.450313	lr=0.000100
Epoch[0] Batch [449]	Speed: 99.513935 samples/sec	accuracy=0.473333	lr=0.000100
Epoch[0] Batch [499]	Speed: 99.618631 samples/sec	accuracy

Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_4/_test_pred_probs_split_4
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_4/_test_pred_features_split_4
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_4/_test_labels_split_4
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_4/_test_image_files_split_4
Saving ./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/split_4/_test_indices_split_4
CPU times: user 8h 37min 42s, sys: 38min 11s, total: 9h 15min 54s
Wall time: 9h 21min 38s


## Read pickle files from cross-validation

In [6]:
def load_pickle(pickle_file_name):
    """Load pickle file"""

    print(f"Loading {pickle_file_name}")

    with open(pickle_file_name, 'rb') as handle:
        out = pickle.load(handle)
        
    return out

In [8]:
out_subfolder = "./cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_resnet18/split_0/"
split_num = 0

get_pickle_file_name = (
    lambda object_name: f"{out_subfolder}_{object_name}_split_{split_num}"
)

pred_probs = load_pickle(get_pickle_file_name("test_pred_probs"))
pred_features = load_pickle(get_pickle_file_name("test_pred_features"))
labels = load_pickle(get_pickle_file_name("test_labels"))
images = load_pickle(get_pickle_file_name("test_image_files"))
indices = load_pickle(get_pickle_file_name("test_indices"))

Loading ../cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_resnet18/split_0/_test_pred_probs_split_0
Loading ../cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_resnet18/split_0/_test_pred_features_split_0
Loading ../cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_resnet18/split_0/_test_labels_split_0
Loading ../cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_resnet18/split_0/_test_image_files_split_0
Loading ../cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_resnet18/split_0/_test_indices_split_0


In [9]:
pred_probs

array([[7.67939985e-01, 2.02408936e-02, 6.41666353e-02, ...,
        1.89649351e-02, 5.00814468e-02, 8.37699976e-03],
       [7.88341939e-01, 5.19996397e-02, 4.19315025e-02, ...,
        1.68199819e-02, 7.51034394e-02, 7.98258651e-03],
       [5.23230553e-01, 6.63620755e-02, 2.74402052e-01, ...,
        2.06029266e-02, 5.82620092e-02, 1.80467870e-02],
       ...,
       [2.86818724e-02, 2.56231993e-01, 1.65443495e-02, ...,
        5.70716860e-04, 2.18237489e-02, 5.69517255e-01],
       [6.93339435e-03, 2.08868161e-02, 8.61072987e-02, ...,
        1.18399109e-03, 2.10009608e-02, 7.91855156e-01],
       [1.90793425e-02, 2.31997311e-01, 1.42989323e-01, ...,
        9.68242437e-03, 5.08847134e-03, 4.59250033e-01]])

## Display GPU used for training

In [ ]:
!nvidia-smi